In [4]:
import matplotlib.pyplot as plt

In [6]:
import cv2
cv2.__version__

'3.4.2'

In [7]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
mypath="C:\\Users\\USER\\data_waffle"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
images_waffle = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    images_waffle[n] = cv2.imread(join(mypath, onlyfiles[n]), cv2.IMREAD_COLOR)
import glob    
for i in range(len(images_waffle)):
    b, g, r = cv2.split(images_waffle[i])
    images_waffle[i] = cv2.merge([r, g, b])

In [8]:
mypath="C:\\Users\\USER\\data_macarons"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
images_macarons = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    images_macarons[n] = cv2.imread(join(mypath, onlyfiles[n]), cv2.IMREAD_COLOR)

In [9]:
waffle_reshape = images_waffle
for i in range(len(images_waffle)):
    waffle_reshape[i] = cv2.resize(images_waffle[i], dsize=(50, 50), interpolation=cv2.INTER_AREA)


In [10]:
macarons_reshape = images_macarons
for i in range(len(images_macarons)):
    macarons_reshape[i] = cv2.resize(images_macarons[i], dsize=(50, 50), interpolation=cv2.INTER_AREA)


In [11]:
waffle_train = waffle_reshape[:800]
waffle_test = waffle_reshape[800:]
print(len(waffle_test))

200


In [12]:
macarons_train = macarons_reshape[:800]
macarons_test = macarons_reshape[800:]
print(len(macarons_test))

200


In [13]:
import numpy as np
train_data = np.concatenate((waffle_train, macarons_train))
test_data = np.concatenate((waffle_test, macarons_test))


In [14]:

train_label = [True for _ in range(800)] + [False for _ in range(800)]
test_label = [True for _ in range(200)] + [False for _ in range(200)]
train_label_np = np.asarray(train_label)
test_label_np = np.asarray(test_label)
train_data_np = np.asarray(train_data)
test_data_np = np.asarray(test_data)

In [15]:
train_data_np_f = np.asarray([dataset.flatten() for dataset in train_data_np])
test_data_np_f = np.asarray([dataset.flatten() for dataset in test_data_np])


In [28]:
import pandas as pd
train_data_np_f_pd = pd.DataFrame(train_data_np_f)
train_data_np_f_pd
train_data_np_f[0].shape

(7500,)

In [14]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(train_data_np_f, train_label_np)
sgd_pred=sgd_clf.predict(test_data_np_f)
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_label_np,sgd_pred)))

예측 정확도: 0.9325


In [15]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, train_data_np_f, train_label_np, cv=3, scoring="accuracy")


array([0.97565543, 0.96998124, 0.98123827])

In [160]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=42)

In [161]:
dt_clf.fit(train_data_np_f, train_label_np)

DecisionTreeClassifier(random_state=42)

In [162]:
pred = dt_clf.predict(test_data_np_f)

from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_label_np,pred)))

예측 정확도: 0.8775


In [163]:
dt_clf_cross=cross_val_score(dt_clf, train_data_np_f, train_label_np, cv=3, scoring="accuracy")
dt_clf_cross


array([0.87078652, 0.85365854, 0.86116323])

In [164]:
from graphviz import Source
from graphviz import render

from sklearn.tree import export_graphviz
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "desert_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "d_images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
export_graphviz(
        dt_clf,
        out_file=os.path.join(IMAGES_PATH, "desert_tree.dot"),
        class_names=['True','False, NONE'],
        rounded=True,
        filled=True
    )

gr=Source.from_file(os.path.join(IMAGES_PATH, "desert_tree.dot"))



In [165]:

gr.render('test3', view=True)  

'test3.pdf'

In [59]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC


In [67]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)

In [69]:
estimators = [random_forest_clf, dt_clf, svm_clf, sgd_clf]

for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(train_data_np_f, train_label_np)

Training the RandomForestClassifier(random_state=42)
Training the DecisionTreeClassifier(random_state=42)
Training the LinearSVC(max_iter=100, random_state=42, tol=20)
Training the SGDClassifier(random_state=42)


In [70]:
[estimator.score(test_data_np_f, test_label_np) for estimator in estimators]

[0.9675, 0.8775, 0.96, 0.9325]

In [172]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("decision_trees_clf", dt_clf),
    ("svm_clf", svm_clf),
    ("sgd_clf", sgd_clf),
]
voting_clf = VotingClassifier(named_estimators,voting='hard')

In [173]:
voting_clf.fit(train_data_np_f, train_label_np)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('decision_trees_clf',
                              DecisionTreeClassifier(random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('sgd_clf', SGDClassifier(random_state=42))])

In [147]:
[estimator.score(test_data_np_f, test_label_np) for estimator in voting_clf.estimators_]

[0.9675, 0.8775, 0.96, 0.9325]

In [167]:
voting_clf.score(test_data_np_f, test_label_np)

0.9725

In [168]:
voting_clf.voting = "soft"

In [170]:
voting_clf

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('decision_trees_clf',
                              DecisionTreeClassifier(random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('sgd_clf', SGDClassifier(random_state=42))],
                 voting='soft')